In [1]:
import re
import json

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing import sequence

In [2]:
DEFAULT_PATH ='./data_in/' # 파일이 저장된 경로

In [3]:
# 데이터 프레임 형태로 학습 데이터를 불러온다.
# header=0 : 파일의 첫 번째에 열 이름이 표시된다.
# delimiter="\t" :  tab을 기준으로 구분한다
# quoting=3 : 쌍따옴표를 무시한다.
train_df = pd.read_csv(DEFAULT_PATH + "labeledTrainData.tsv", header=0, delimiter="\t", quoting=3) 

In [4]:
def preprocessing( review, remove_stopwords = False ): 
    # 불용어 제거는 옵션으로 선택 가능하다.
    
    # 1. HTML 태그 제거
    review_text = BeautifulSoup(review, "html5lib").get_text()

    # 2. 영어가 아닌 특수문자들을 공백(" ")으로 바꾸기
    review_text = re.sub("[^a-zA-Z]", " ", review_text)

    # 3. 대문자들을 소문자로 바꾸고 공백단위로 텍스트들 나눠서 리스트로 만든다.
    words = review_text.lower().split()
    
    if remove_stopwords: 
        # 4. 불용어들을 제거
    
        #영어에 관련된 불용어 불러오기
        stops = set(stopwords.words("english"))
        # 불용어가 아닌 단어들로 이루어진 새로운 리스트 생성
        words = [w for w in words if not w in stops]
        # 5. 단어 리스트를 공백을 넣어서 하나의 글로 합친다.
        clean_review = ' '.join(words)
        
    else: # 불용어 제거하지 않을 때
        clean_review = ' '.join(words)

    return clean_review

In [5]:
#train데이터를 전처리
clean_train_reviews = []

for review in train_df['review']:
    clean_train_reviews.append(preprocessing(review, remove_stopwords = True))

In [6]:
DATA_IN_PATH = './data_in/'

TRAIN_INPUT_DATA = 'train_input.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
TEST_INPUT_DATA = 'test_input.npy'

DATA_CONFIGS = 'data_configs.json'

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_reviews)
text_sequences = tokenizer.texts_to_sequences(clean_train_reviews)

word_vocab = tokenizer.word_index # 딕셔너리 형태
print("전체 단어 개수: ", len(word_vocab)) # 전체 단어 개수 확인


###TODO2: 문장 최대길이를 전체 문장의 median으로 변경 필요

MAX_SEQUENCE_LENGTH = 174 # 문장 최대 길이

inputs = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post') # 문장의 길이가 50 단어가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
labels = np.array(train_df['sentiment']) # 각 리뷰의 감정을 넘파이 배열로 만든다.

print('Shape of input data tensor:', inputs.shape) # 리뷰 데이터의 형태 확인
print('Shape of label tensor:', labels.shape) # 감정 데이터 형태 확인

전체 단어 개수:  74065
Shape of input data tensor: (25000, 174)
Shape of label tensor: (25000,)


In [8]:
data_configs = {}
data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab) # vocab size 추가

import os
# 저장하는 디렉토리가 존재하지 않으면 생성
if not os.path.exists(DEFAULT_PATH + DATA_IN_PATH):
    os.makedirs(DEFAULT_PATH + DATA_IN_PATH)
# 전처리 된 데이터를 넘파이 형태로 저장
np.save(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'wb'), inputs)
np.save(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'wb'), labels)

if not os.path.exists(DATA_IN_PATH):
    os.makedirs(DATA_IN_PATH)

# 데이터 사전을 json 형태로 저장
json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'), ensure_ascii=False)

In [9]:
test_data = pd.read_csv(DEFAULT_PATH + "testData.tsv", header=0, delimiter="\t", quoting=3)

clean_test_reviews = []
for review in test_data['review']:
    clean_test_reviews.append(preprocessing(review, remove_stopwords = True))

text_sequences = tokenizer.texts_to_sequences(clean_test_reviews)

test_inputs = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')


import os
# 저장하는 디렉토리가 존재하지 않으면 생성
if not os.path.exists(DATA_IN_PATH):
    os.makedirs(DATA_IN_PATH)

np.save(open(DATA_IN_PATH + TEST_INPUT_DATA, 'wb'), test_inputs)